In [5]:
import torch_geometric
import torch
from torch_geometric.datasets import OGB_MAG
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader

In [26]:
torch.__version__

'1.7.1'

In [2]:
torch_geometric.__version__

'2.0.1'

In [10]:
dataset = OGB_MAG(root='./data', preprocess='transe', transform=T.ToUndirected(merge=True))
data = dataset[0]
print(data)

Using existing file mag_transe_emb.zip
Extracting data/mag/raw/mag_transe_emb.zip
Processing...
Done!


HeteroData(
  paper={
    x=[736389, 128],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 256] },
  field_of_study={ x=[59965, 256] },
  institution={ x=[8740, 256] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)


In [11]:
data.metadata()

(['paper', 'author', 'field_of_study', 'institution'],
 [('author', 'affiliated_with', 'institution'),
  ('author', 'writes', 'paper'),
  ('paper', 'cites', 'paper'),
  ('paper', 'has_topic', 'field_of_study'),
  ('institution', 'rev_affiliated_with', 'author'),
  ('paper', 'rev_writes', 'author'),
  ('field_of_study', 'rev_has_topic', 'paper')])

In [12]:
data.x_dict

{'paper': tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.0616, -0.0277, -0.1338],
         [-0.1510, -0.1073, -0.2220,  ...,  0.3458, -0.0277, -0.2185],
         [-0.1148, -0.1760, -0.2606,  ...,  0.1731, -0.1564, -0.2780],
         ...,
         [ 0.0228, -0.0865,  0.0981,  ..., -0.0547, -0.2077, -0.2305],
         [-0.2891, -0.2029, -0.1525,  ...,  0.1042,  0.2041, -0.3528],
         [-0.0890, -0.0348, -0.2642,  ...,  0.2601, -0.0875, -0.5171]]),
 'author': tensor([[ 0.3226,  0.0584, -0.3172,  ..., -0.1393,  0.3022, -0.8178],
         [ 0.0438, -0.1213,  0.1804,  ..., -0.3374,  0.4846, -0.4272],
         [-0.2300, -0.3192,  0.1979,  ..., -0.1007,  1.3342, -0.4569],
         ...,
         [-0.5160,  0.1867,  0.2854,  ..., -0.1987, -0.6515,  0.5417],
         [-0.3726, -0.1563, -0.1685,  ...,  0.1218,  0.4354, -0.0998],
         [-0.7412,  0.5156,  0.0240,  ..., -0.2002, -0.2625, -0.1643]]),
 'field_of_study': tensor([[-0.5958,  0.4260,  0.0320,  ..., -0.2224,  0.2888, -0.3681],
        

In [13]:
data.edge_index_dict

{('author',
  'affiliated_with',
  'institution'): tensor([[      0,       1,       2,  ..., 1134645, 1134647, 1134648],
         [    845,     996,    3197,  ...,    5189,    4668,    4668]]),
 ('author',
  'writes',
  'paper'): tensor([[      0,       0,       0,  ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768,  ...,  657395,  671118,  719594]]),
 ('paper',
  'cites',
  'paper'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [    88,  27449, 121051,  ..., 707740, 732008, 732389]]),
 ('paper',
  'has_topic',
  'field_of_study'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [   145,   2215,   3205,  ...,  21458,  22283,  31934]]),
 ('institution',
  'rev_affiliated_with',
  'author'): tensor([[    845,     996,    3197,  ...,    5189,    4668,    4668],
         [      0,       1,       2,  ..., 1134645, 1134647, 1134648]]),
 ('paper',
  'rev_writes',
  'author'): tensor([[  19703,  289285,  311768,  ...,  657

In [15]:
train_loader = NeighborLoader(data, num_neighbors=[15, 15], batch_size=128, input_nodes=('paper', data['paper'].train_mask), shuffle=True)

In [16]:
print(train_loader)
batch = next(iter(train_loader))
print(batch)

NeighborLoader()
HeteroData(
  paper={
    x=[23472, 128],
    y=[23472],
    train_mask=[23472],
    val_mask=[23472],
    test_mask=[23472],
    batch_size=128
  },
  author={ x=[4564, 256] },
  field_of_study={ x=[2992, 256] },
  institution={ x=[345, 256] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 6007] },
  (paper, cites, paper)={ edge_index=[2, 13508] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11326] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 845] },
  (paper, rev_writes, author)={ edge_index=[2, 5145] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 11970] }
)


In [17]:
from torch_geometric.nn import TransformerConv, GCNConv, GATConv, SAGEConv, to_hetero
import torch.nn as nn
import torch.nn.functional as F 

In [18]:
class Net1(torch.nn.Module):
    def __init__(self, hidden_dim, num_classes, num_layers=2) -> None:
        super(Net1, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.num_layers = num_layers

        self.convs = nn.ModuleList()
        self.lins = nn.ModuleList()
        self.bns = nn.ModuleList()

        for i in range(self.num_layers):
            self.convs.append(GATConv((-1, -1), hidden_dim))
            self.lins.append(nn.Linear(hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        # self.dropout = torch.nn.Dropout()
        self.fc_out = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index) + self.lins[i](x)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc_out(x)
        return x

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = Net1(hidden_dim=64, num_classes=dataset.num_classes, num_layers=2)
model = to_hetero(model, data.metadata(), aggr='sum').to(device)
print(model)

cuda


AttributeError: module 'torch.nn.parameter' has no attribute 'UninitializedParameter'

In [ ]:
@torch.no_grad()
def init_params():
    # Initialize lazy parameters via forwarding a single batch to the model:
    batch = next(iter(train_loader))
    batch = batch.to(device)
    model(batch.x_dict, batch.edge_index_dict)

In [1]:
import torch

In [3]:
emb = torch.load('./data/mag/raw/mag_metapath2vec_emb.pt', map_location='cpu')
emb['paper'].shape

torch.Size([736389, 128])

In [7]:
emb['author'].shape

torch.Size([1134649, 128])

In [9]:
d1 = {'1': torch.rand((2, 3)), '2': torch.rand((4, 3))}
d2 = {'1': torch.rand((2, 3)), '2': torch.rand((4, 3))}
{key: x + d2[key] for key, x in d1.items()}

{'1': tensor([[1.8597, 0.6279, 1.0817],
         [0.5914, 0.7982, 0.2904]]),
 '2': tensor([[0.7516, 1.5576, 1.0026],
         [0.9337, 1.3418, 1.0792],
         [0.8797, 0.9528, 0.7635],
         [1.3031, 0.7651, 1.3356]])}